In [1]:
##################################
### Import the usual libraries ###
##################################

### Other
import warnings
warnings.filterwarnings('ignore')

### Matplotlib
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rcParams.update({'font.size': 30})

In [2]:
import numpy as np
import astropy
from astropy.io import fits
from astropy.table import Table
from scipy.interpolate import InterpolatedUnivariateSpline

def closest(data,value):
    '''
    Find nearest value in array to given value
        
    Inputs:
    ------
        data: data to search through 
        value: value of interest
        
    Output:
    ------
        close_value
    '''
    
    value = np.asarray(value)
    data = np.asarray(data)
    
    close_value = data[(np.abs(np.subtract(data,value))).argmin()]
    return close_value

def closest2(data,value):
    '''
    Find values of two elements closest to the given value
    
    Inputs:
    ------
        data: data to search through 
        value: value of interest
        
    Output:
    ------
        close1: closest value under the given value
        close2: closest value over the given value
    '''
    
    value = np.asarray(value)
    data = np.asarray(data)
    
    close1 = data[(np.abs(np.subtract(data,value))).argmin()]
    
    data = data[np.where(data!=close1)]
    
    close2 = data[(np.abs(np.subtract(data,value))).argmin()]
    
    return close1,close2

In [74]:
class WhatsMyAgeAgain():
    '''
    Class to calculate a star's age and extinction using PARSEC isochrones and extinction law
    from Cardelli et al. 1989
    '''
    def __init__(self,feh,am,distance,isochrones):
        
        '''
        feh: [float] metallicity of star
        am: [float] [$\alpha$/M] of star
        distance: [float] distance to star in pc
        isochrones: [astropy Table] PARSEC isochrone table
        '''
        
        # stellar parameters
        self.feh = feh
        self.salfeh = feh+np.log10(0.638*(10**am)+0.362) #Salaris et al. 1993
        self.dist = distance
        
        # PARSEC isochrones
        self.j = 'Jmag'
        self.h = 'Hmag'
        self.k = 'Ksmag'
        self.uniq_ages = 10**np.unique(isochrones['logAge'])/10**9
        self.iso = isochrones
        
        # Effective Wavelengths of different passbands
        self.leff = {'BP':0.5387,'G':0.6419,'RP':0.7667,'J':1.2345,'H':1.6393,'K':2.1757} # mircons
        
    def car_a(self,x):
        '''
        a(x) function from Cardelli et al. 1989
    
        Input:
        -----
            x: effective wavelength in units of 1/micron
        
        Output:
        ------
            a: a function value 
        '''
        
        if 0.3 <= x < 1.1:
            a = 0.574*(x**1.61)
            return a
    
        elif 1.1 <= x < 3.3:
            y = x - 1.82
            a = (1.+0.17699*y-0.50477*(y**2)-0.02427*(y**3)+0.72085*(y**4)+
                 0.01979*(y**5)-0.77530*(y**6)+0.32999*(y**7))
            return a
    
        elif 3.3 <= x < 8.0:
            if x < 5.9:
                a = 1.752-0.136*x-0.104/((x-4.67)**2+0.341)
                return a
        
            else:
                fa = -0.04473*((x-5.9)**2)+0.1207*((x-5.9)**3)
                a = 1.752-0.136*x-0.104/((x-4.67)**2+0.341)+fa
                return a
            
    def car_b(self,x):
        '''
        b(x) function from Cardelli et al. 1989
    
        Input:
        -----
            x: effective wavelength in units of 1/micron
        
        Output:
        ------
            b: b function value 
        '''
        
        if 0.3 <= x < 1.1:
            b = -0.527*(x**1.61)
            return b
    
        elif 1.1 <= x <= 3.3:
            y = x - 1.82
            b = (1.41338*y+2.28305*(y**2)+1.07233*(y**3)-5.38434*(y**4)-
                 0.62251*(y**5)+5.30260*(y**6)-2.09002*(y**7))
            return b
    
        elif 3.3 <= x < 8.0:
            if x < 5.9:
                b = -3.090+1.825*x+1.206/((x-4.62)**2+0.263)
                return b
        
            else:
                fb = 0.2130*((x-5.9)**2)+0.1207*((x-5.9)**3)
                b = -3.090+1.825*x+1.206/((x-4.62)**2+0.263)+fb
                return b
            
    def car_alak(self,wave):
        '''
        Calculate the relative extinction with the K band as the fiducial band and
        Rv (=Av/E(B_V)) = 3.1
        
        Input:
        -----
            wave: effective wavelength of band in microns
        
        Output:
        ------
            alak: extinction relative to the K band
        '''
        
        x=1/wave
        alak = (3.1*self.car_a(x)+self.car_b(x))/(3.1*self.car_a(1/2.1757)+self.car_b(1/2.1757))
        
        return alak
    
    def ind_alak(self,wave):
        '''
        Calculate the relative extinction to Ak using Indebetouw et al. 2005.
        This uses GLIMPSE data and has only been verified in IR.
    
        Input:
        -----
            wave: effective wavelength in microns of the passband
        
        Output:
        ------
            alak: A\lambda/Ak
        '''
        # 0.61 +/- 0.04; -2.22 +/- 0.17; 1.21 +/- 0.23
        alak = 10**(0.61-2.22*np.log10(wave)+1.21*(np.log10(wave)**2))
        return alak
        
    def Teff2AppMags(self,teff,age,ak):
        '''
        Calculate the expected apparent magnitude of a star
        
        Inputs:
        ------
            teff: temperature of star
            age: age of star
            al: extinction in the same band used to initialize the class
            
            # parameters defined in __init__
            self.dist: distance to star
            self.iso: set of PARSEC isochrones
        
        Output:
        ------
            calc_mag: expected intrinsic magnitude for the given temperature
        '''
        
        lgteff = np.log10(teff)
        
        # Figure out if age is actually in the ages given in the isochrone table
        if age in self.uniq_ages:
            
            print('in: ',age)
            
            # pick out a single isochrone 
            iso_ = self.iso[np.where((self.iso['logAge']==closest(self.iso['logAge'],np.log10(age*10**9)))&
                                       (self.iso['MH']==closest(self.iso['MH'],np.mean(self.salfeh))))]
            
            # sort so temp is always increasing
            sidx = np.argsort(iso_['logTe'])
            slogTe = iso_['logTe'][sidx]
            
            sj = iso_[self.j][sidx]
            sh = iso_[self.h][sidx]
            sk = iso_[self.k][sidx]
            
            # splines for each apparent magnitude as function of teff
            _, uidx = np.unique(slogTe,return_index=True)
            
            j_spl = InterpolatedUnivariateSpline(slogTe[uidx],sj[uidx])
            h_spl = InterpolatedUnivariateSpline(slogTe[uidx],sh[uidx])
            k_spl = InterpolatedUnivariateSpline(slogTe[uidx],sk[uidx])
            
            j_app = j_spl(lgteff)+5.0*np.log10(self.dist)-5.0+ak*self.ind_alak(self.leff['J'])
            h_app = h_spl(lgteff)+5.0*np.log10(self.dist)-5.0+ak*self.ind_alak(self.leff['H'])
            k_app = k_spl(lgteff)+5.0*np.log10(self.dist)-5.0+ak
            
            print('Calculated Apparent J: {:.3f} H: {:.3f} K: {:.3f}'.format(j_app,h_app,k_app))
            print('ak: {:.3f}'.format(ak))
            print('---')
            
            return j_app,h_app,k_app
            
        else:
            age_lo,age_hi = closest2(self.uniq_ages,age)
            
            print('not in: ',age)
            
            ### age_lo ###
            # pick out a single isochrone 
            iso_lo = self.iso[np.where((self.iso['logAge']==closest(self.iso['logAge'],np.log10(age_lo*10**9)))&
                                       (self.iso['MH']==closest(self.iso['MH'],np.mean(self.salfeh))))]
            
            # sort so temp is always increasing
            sidx_lo = np.argsort(iso_lo['logTe'])
            slogTe_lo = iso_lo['logTe'][sidx_lo]
            
            sj_lo = iso_lo[self.j][sidx_lo]
            sh_lo = iso_lo[self.h][sidx_lo]
            sk_lo = iso_lo[self.k][sidx_lo]
            
            # splines for each magnitude
            _, uidx_lo = np.unique(slogTe_lo,return_index=True)
            
            j_spl_lo = InterpolatedUnivariateSpline(slogTe_lo[uidx_lo],sj_lo[uidx_lo])
            h_spl_lo = InterpolatedUnivariateSpline(slogTe_lo[uidx_lo],sh_lo[uidx_lo])
            k_spl_lo = InterpolatedUnivariateSpline(slogTe_lo[uidx_lo],sk_lo[uidx_lo])
            
            ### age_hi ####
            # pick out a single isochrone 
            iso_hi = self.iso[np.where((self.iso['logAge']==closest(self.iso['logAge'],np.log10(age_hi*10**9)))&
                                       (self.iso['MH']==closest(self.iso['MH'],np.mean(self.salfeh))))]
            
            # sort so temp is always increasing
            sidx_hi = np.argsort(iso_hi['logTe'])
            slogTe_hi = iso_hi['logTe'][sidx_hi]
            
            sj_hi = iso_hi[self.j][sidx_hi]
            sh_hi = iso_hi[self.h][sidx_hi]
            sk_hi = iso_hi[self.k][sidx_hi]
            
            # splines for each magnitude as function of log(teff)
            _, uidx_hi = np.unique(slogTe_hi,return_index=True)
            
            j_spl_hi = InterpolatedUnivariateSpline(slogTe_hi[uidx_hi],sj_hi[uidx_hi])
            h_spl_hi = InterpolatedUnivariateSpline(slogTe_hi[uidx_hi],sh_hi[uidx_hi])
            k_spl_hi = InterpolatedUnivariateSpline(slogTe_hi[uidx_hi],sk_hi[uidx_hi])
            
            ### Interpolate Between hi and lo ###
            # InterpolatedUnivariateSpline does not work with just two points
            j_spl_interp = np.poly1d(np.polyfit([age_lo,age_hi],[j_spl_lo(lgteff),j_spl_hi(lgteff)],1))
            h_spl_interp = np.poly1d(np.polyfit([age_lo,age_hi],[h_spl_lo(lgteff),h_spl_hi(lgteff)],1))
            k_spl_interp = np.poly1d(np.polyfit([age_lo,age_hi],[k_spl_lo(lgteff),k_spl_hi(lgteff)],1))
            
            # Calculate the theoretical apparent magnitude
            j_app = j_spl_interp(age)+5.0*np.log10(self.dist)-5.0+ak*self.ind_alak(self.leff['J'])
            h_app = h_spl_interp(age)+5.0*np.log10(self.dist)-5.0+ak*self.ind_alak(self.leff['H'])
            k_app = k_spl_interp(age)+5.0*np.log10(self.dist)-5.0+ak
            
            print('Calculated Apparent J: {:.3f} H: {:.3f} K: {:.3f}'.format(j_app,h_app,k_app))
            print('ak: {:.3f}'.format(ak))
            print('---')
            
            return j_app,h_app,k_app
        
#     def MCDistribution(self,quant,qaunt_err,number):
#         '''
#         Generate a distribution to do a Monte Simulation
            
#         Inputs:
#         ------
#             quant: value to creat distribution for
#             quant_err: error in quant
#             number: number of points to sample
              
#         Outputs:
#         -------
#             noisy: normal distribution about quant with sigma of quant_err
#         '''
            
#         noisy = np.random.normal(quant,quant_err,number)
            
#         return noisy

# Parsec

In [6]:
# massive
massive = fits.getdata('/Users/joshuapovick/Desktop/Research/parsec/parsec_massive.fits.gz',0)
massive = massive[np.where(massive['label']==3.0)]

# Globular Clusters

In [7]:
### GCS Data
gcs = fits.getdata('/Users/joshuapovick/Desktop/Research/fits/allStar-r13-l33-58932beta_apa_dist_galvel_gc.fits.gz')
cln = np.where((gcs['FE_H']>-9999.0)&(gcs['AK_TARG']>-9999.0)&(gcs['LOGG']>0.0)&(gcs['M_H_ERR']>-90.0)&
                (gcs['C_FE']>-9999.0)&(gcs['N_FE']>-9999.0))
gcs = Table(gcs[cln])

### Find Cluster with more than one star

idx = []
for i in range(len(np.unique(gcs['CLUSTER']))):
    idx.append(np.squeeze(np.where(gcs['CLUSTER']==np.unique(gcs['CLUSTER'])[i])))

for i in range(len(idx)):
    try:
        len(idx[i])
    except:
        print('bad: ',i)
        
good_names = []
for i in np.asarray(idx)[np.delete(np.asarray(list(range(len(idx)))),[24,27,37])]:
    if len(i)>10:
        good_names.append(gcs['CLUSTER'][i][0])

print(good_names)

good_clus = np.where((gcs['CLUSTER']=='47Tuc')|(gcs['CLUSTER']=='M10')|(gcs['CLUSTER']=='M107')|
                     (gcs['CLUSTER']=='M12')|(gcs['CLUSTER']=='M13')|(gcs['CLUSTER']=='M19')|
                     (gcs['CLUSTER']=='M2')|(gcs['CLUSTER']=='M22')|(gcs['CLUSTER']=='M3')|
                     (gcs['CLUSTER']=='M4')|(gcs['CLUSTER']=='M5')|(gcs['CLUSTER']=='M53')|
                     (gcs['CLUSTER']=='M54')|(gcs['CLUSTER']=='M55')|(gcs['CLUSTER']=='M71')|
                     (gcs['CLUSTER']=='M79')|(gcs['CLUSTER']=='NGC1851')|(gcs['CLUSTER']=='NGC2808')|
                     (gcs['CLUSTER']=='NGC288')|(gcs['CLUSTER']=='NGC3201')|(gcs['CLUSTER']=='NGC362')|
                     (gcs['CLUSTER']=='NGGC6388')|(gcs['CLUSTER']=='NGC6397')|(gcs['CLUSTER']=='NGC6752')|
                     (gcs['CLUSTER']=='omegaCen'))

gcs = gcs[good_clus]

bad:  24
bad:  27
bad:  37
['47Tuc', 'M10', 'M107', 'M12', 'M13', 'M19', 'M2', 'M22', 'M3', 'M4', 'M5', 'M53', 'M54', 'M55', 'M71', 'M79', 'NGC1851', 'NGC2808', 'NGC288', 'NGC3201', 'NGC362', 'NGC6388', 'NGC6397', 'NGC6752', 'omegaCen']


# Test Code

In [8]:
from scipy.optimize import curve_fit

In [75]:
# pick star
g_idx = 1000

# initialize class
CalcAge = WhatsMyAgeAgain(gcs['M_H'][g_idx],gcs['ALPHA_M'][g_idx],10400,massive)

# curve_fit does its thing
poptk, pcovk = curve_fit(CalcAge.Teff2AppMags,gcs['TEFF'][g_idx],
                         np.array([[gcs['J'][g_idx],gcs['H'][g_idx],gcs['K'][g_idx]]],dtype=float).T.ravel(),
                         p0=[12.,10.],bounds=((0.,0.),(14.,np.inf)),method='trf')

# print measured apparent K
print('Measured Apparent J: {:3f} H: {:.3f} K: {:.3f}'.format(gcs['J'][g_idx],gcs['H'][g_idx],gcs['K'][g_idx]))
print('ak: {:.3f}'.format(gcs['AK_TARG'][g_idx]))

not in:  12.0
Calculated Apparent J: 37.855 H: 26.573 K: 21.017
ak: 10.000
---
not in:  12.000000178813934
Calculated Apparent J: 37.855 H: 26.573 K: 21.017
ak: 10.000
---
not in:  12.0
Calculated Apparent J: 37.855 H: 26.573 K: 21.017
ak: 10.000
---
not in:  6.086285803088214
Calculated Apparent J: 24.467 H: 18.530 K: 15.710
ak: 4.988
---
not in:  6.08628589378094
Calculated Apparent J: 24.467 H: 18.530 K: 15.710
ak: 4.988
---
not in:  6.086285803088214
Calculated Apparent J: 24.467 H: 18.530 K: 15.710
ak: 4.988
---
not in:  3.111598463807708
Calculated Apparent J: 17.756 H: 14.431 K: 12.949
ak: 2.538
---
not in:  3.1115985101741384
Calculated Apparent J: 17.756 H: 14.431 K: 12.949
ak: 2.538
---
not in:  3.111598463807708
Calculated Apparent J: 17.756 H: 14.431 K: 12.949
ak: 2.538
---
not in:  1.6988670300328772
Calculated Apparent J: 14.333 H: 12.276 K: 11.443
ak: 1.351
---
not in:  1.6988670553479686
Calculated Apparent J: 14.333 H: 12.276 K: 11.443
ak: 1.351
---
not in:  1.69886703

Calculated Apparent J: 11.560 H: 10.884 K: 10.760
ak: 0.053
---
not in:  5.850036062842721
Calculated Apparent J: 11.560 H: 10.884 K: 10.760
ak: 0.053
---
not in:  5.850073685060851
Calculated Apparent J: 11.560 H: 10.884 K: 10.759
ak: 0.053
---
not in:  5.850045468410027
Calculated Apparent J: 11.560 H: 10.884 K: 10.760
ak: 0.053
---
not in:  5.850038414235268
Calculated Apparent J: 11.560 H: 10.884 K: 10.760
ak: 0.053
---
not in:  5.850038501407633
Calculated Apparent J: 11.560 H: 10.884 K: 10.760
ak: 0.053
---
not in:  5.850038414235268
Calculated Apparent J: 11.560 H: 10.884 K: 10.760
ak: 0.053
---
not in:  5.85004311701873
Calculated Apparent J: 11.560 H: 10.884 K: 10.760
ak: 0.053
---
not in:  5.850039589931313
Calculated Apparent J: 11.560 H: 10.884 K: 10.760
ak: 0.053
---
not in:  5.850039677103696
Calculated Apparent J: 11.560 H: 10.884 K: 10.760
ak: 0.053
---
not in:  5.850039589931313
Calculated Apparent J: 11.560 H: 10.884 K: 10.760
ak: 0.053
---
not in:  5.850041941322996
